In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import cmip6_cv

In [12]:
help(cmip6_cv)

Help on package cmip6_cv:

NAME
    cmip6_cv

PACKAGE CONTENTS
    PrePARE (package)
    _cmip6_cv
    cmor_const
    pywrapper

DATA
    CMOR_AXIS_ENTRY_FILE = '_AXIS_ENTRY_FILE'
    CMOR_CF_VERSION_MAJOR = 1
    CMOR_CF_VERSION_MINOR = 7
    CMOR_CRITICAL = 22
    CMOR_DEFAULT_FILE_TEMPLATE = None
    CMOR_DEFAULT_FURTHERURL_TEMPLATE = 'https://furtherinfo.es-doc.org/<mi...
    CMOR_DEFAULT_PATH_TEMPLATE = None
    CMOR_EXIT = 1
    CMOR_EXIT_ON_MAJOR = 0
    CMOR_EXIT_ON_WARNING = 2
    CMOR_FORMULA_VAR_FILE = '_FORMULA_VAR_FILE'
    CMOR_MAX_ATTRIBUTES = 100
    CMOR_MAX_AXES = 1500
    CMOR_MAX_DIMENSIONS = 7
    CMOR_MAX_ELEMENTS = 500
    CMOR_MAX_ERRORS = 10
    CMOR_MAX_GRIDS = 100
    CMOR_MAX_GRID_ATTRIBUTES = 25
    CMOR_MAX_STRING = 1024
    CMOR_MAX_TABLES = 30
    CMOR_MAX_VARIABLES = 500
    CMOR_NORMAL = 21
    CMOR_QUIET = 0
    CMOR_VERSION_MAJOR = 3
    CMOR_VERSION_MINOR = 7
    CMOR_VERSION_PATCH = 0
    CMOR_WARNING = 20
    FILE_NAME_TEMPLATE = 'output_file_temp

In [54]:
import os

import cordex as cx
import xarray as xr
from cordex import cmor as cmor

table_dir = "./Tables"

cmor.set_options(table_prefix="CORDEX")


def test_cmorizer_fx():
    ds = cx.cordex_domain("EUR-11", dummy="topo")
    filename = cmor.cmorize_variable(
        ds,
        "orog",
        mapping_table={"orog": {"varname": "topo"}},
        cmor_table=os.path.join(table_dir, "CORDEX_fx.json"),
        dataset_table=os.path.join(table_dir, "CORDEX_remo_example.json"),
        grids_table=os.path.join(table_dir, "CORDEX_grids.json"),
        CORDEX_domain="EUR-11",
        time_units=None,
        allow_units_convert=True,
    )
    return filename

In [55]:
f = test_cmorizer_fx()
f

/Users/buntemey/python/packages/py-cordex/cordex/core/domain.py:632: UserWarning: Order of vertices has changed since v0.3.2 to CF Conventions, see https://github.com/euro-cordex/py-cordex/issues/34
  warn(


'CORDEX/CMIP6/CORDEX/EUR-11/GERICS/REMO2020/ECMWF/ECMWF-ERA5/evaluation/r1i1p1f1/fx/orog/gn/v20221111/orog_EUR-11_ECMWF-ERA5_evaluation_r1i1p1f1_GERICS_REMO2020_v1_fx.nc'

In [51]:
xr.open_dataset(f)

<xarray.Dataset>
Dimensions:                     (rlat: 412, rlon: 424, vertices: 4)
Coordinates:
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.73 21.84
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.05 18.16
    latitude                    (rlat, rlon) float64 ...
    longitude                   (rlat, rlon) float64 ...
Dimensions without coordinates: vertices
Data variables:
    rotated_latitude_longitude  int32 ...
    vertices_latitude           (rlat, rlon, vertices) float64 ...
    vertices_longitude          (rlat, rlon, vertices) float64 ...
    orog                        (rlat, rlon) float32 ...
Attributes: (12/43)
    CORDEX_domain:            Europe
    CORDEX_domain_id:         EUR-11
    Conventions:              CF-1.7 CMIP-6.2
    activity_id:              CORDEX
    contact:                  gerics-cordex2@hereon.de
    creation_date:            2022-11-11T19:05:01Z
    ...                       ...
    title:                    REMO2020 output prepared for CMIP6
    tracking_id:              hdl:21.14100/d70b1e10-294f-4ee9-8aff-9991428b02c1
    variable_id:              orog
    variant_label:            r1i1p1f1
    license:                  REMO2020 data produced by GERICS is licensed un...
    cmor_version:             3.7.0

In [52]:
!ncdump -h $f

netcdf orog_EUR-11_ECMWF-ERA5_evaluation_r1i1p1f1_GERICS_REMO2020_v1_fx {
dimensions:
	rlat = 412 ;
	rlon = 424 ;
	bnds = 2 ;
	vertices = 4 ;
variables:
	double rlat(rlat) ;
		rlat:units = "degrees" ;
		rlat:axis = "Y" ;
		rlat:long_name = "latitude in rotated pole grid" ;
		rlat:standard_name = "grid_latitude" ;
	double rlon(rlon) ;
		rlon:units = "degrees" ;
		rlon:axis = "X" ;
		rlon:long_name = "longitude in rotated pole grid" ;
		rlon:standard_name = "grid_longitude" ;
	int rotated_latitude_longitude ;
		rotated_latitude_longitude:grid_mapping_name = "rotated_latitude_longitude" ;
		rotated_latitude_longitude:grid_north_pole_latitude = 39.25 ;
		rotated_latitude_longitude:grid_north_pole_longitude = -162. ;
		rotated_latitude_longitude:north_pole_grid_longitude = 0. ;
	double latitude(rlat, rlon) ;
		latitude:standard_name = "latitude" ;
		latitude:long_name = "latitude" ;
		latitude:units = "degrees_north" ;
		latitude:missing_value = 1.e+20 ;
		latitude:_FillValue = 1.e+20 ;
		l

In [7]:
from cfchecker.cfchecks import CFChecker

In [8]:
checker = CFChecker()

In [10]:
res = checker.checker(f)

CHECKING NetCDF FILE: CORDEX/CMIP6/CORDEX/GERICS/REMO2020/ECMWF/ECMWF-ERA5/evaluation/r1i1p1f1/fx/orog/gn/v20221011/orog_fx_REMO2020_ECMWF-ERA5_evaluation_r1i1p1f1_gn.nc
Using CF Checker Version 4.1.0
Checking against CF Version CF-1.8
Using Standard Name Table Version 79 (2022-03-19T15:25:54Z)
Using Area Type Table Version 10 (23 June 2020)
Using Standardized Region Name Table Version 4 (18 December 2018)

WARN: (2.6.1): Inconsistency - This netCDF file appears to contain CF-1.7 data, but you've requested a validity check against CF-1.8
WARN: (7.1): Boundary var vertices_latitude should not have attribute units
WARN: (7.1): Boundary var vertices_longitude should not have attribute units

------------------
Checking variable: rlat
------------------

------------------
Checking variable: rlon
------------------

------------------
Checking variable: rotated_latitude_longitude
------------------

------------------
Checking variable: latitude
------------------

------------------
Check

In [19]:
assert not res["global"]["ERROR"]

In [20]:
import json

with open("Tables/CORDEX-CMIP6_CV.json") as f:
    data = json.load(f)

In [24]:
data["CV"].keys()

dict_keys(['required_global_attributes', 'version_metadata', 'activity_id', 'institution_id', 'driving_institution_id', 'source_id', 'driving_source_id', 'source_type', 'frequency', 'grid_label', 'nominal_resolution', 'realm', 'table_id', 'license', 'mip_era', 'experiment_id', 'product', 'tracking_id', 'further_info_url', 'realization_index', 'variant_label', 'data_specs_version', 'Conventions', 'forcing_index', 'initialization_index', 'physics_index'])

In [41]:
for key in data["CV"]:
    filename = "CORDEX-CMIP6_" + key + ".json"
    out = {key: data["CV"][key]}
    with open(filename, "w") as fp:
        json.dump(out, fp, indent=4)
    # print(data["CV"].get(key))

In [28]:
data["CV"]["required_global_attributes"]

['Conventions',
 'CORDEX_domain',
 'activity_id',
 'creation_date',
 'data_specs_version',
 'driving_experiment_id',
 'driving_institution_id',
 'driving_source_id',
 'driving_variant_label',
 'experiment_id',
 'forcing_index',
 'frequency',
 'further_info_url',
 'grid',
 'grid_label',
 'initialization_index',
 'institution',
 'institution_id',
 'license',
 'nominal_resolution',
 'physics_index',
 'product',
 'realization_index',
 'realm',
 'source',
 'source_id',
 'source_type',
 'table_id',
 'tracking_id',
 'variable_id',
 'variant_label']

In [39]:
data["CV"]["activity_id"]

{'CORDEX': 'Coordinated Regional Climate Downscaling Experiment'}

In [116]:
ds = cx.cordex_domain("EUR-11", dummy=True)
ds.rlon.attrs = {}
ds.dummy

<xarray.DataArray 'dummy' (rlat: 412, rlon: 424)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
  * rlon     (rlon) float64 -28.38 -28.27 -28.16 -28.05 ... 17.93 18.05 18.16
  * rlat     (rlat) float64 -23.38 -23.27 -23.16 -23.05 ... 21.61 21.73 21.84
    lon      (rlat, rlon) float64 -10.06 -9.964 -9.864 ... 64.55 64.76 64.96
    lat      (rlat, rlon) float64 21.99 22.03 22.07 22.11 ... 66.81 66.75 66.69
Attributes:
    grid_mapping:  rotated_latitude_longitude
    coordinates:   lat lon

In [105]:
ds.cf["X"]

<xarray.DataArray 'rlon' (rlon: 424)>
array([-28.375, -28.265, -28.155, ...,  17.935,  18.045,  18.155])
Coordinates:
  * rlon     (rlon) float64 -28.38 -28.27 -28.16 -28.05 ... 17.93 18.05 18.16
Attributes:
    axis:           X
    standard_name:  grid_longitude
    long_name:      longitude in rotated pole grid
    units:          degrees

In [118]:
import pandas as pd

pd.to_datetime("2000-01-24T12:00:00")

Timestamp('2000-01-24 12:00:00')

In [121]:
ds = 

AttributeError: 'Dataset' object has no attribute 'time'

In [122]:
def test_cmorizer_mon():
    ds = cx.tutorial.open_dataset("remo_EUR-11_TEMP2_mon")
    filename = cmor.cmorize_variable(
        ds,
        "tas",
        mapping_table={"tas": {"varname": "TEMP2"}},
        cmor_table=os.path.join(table_dir, "CORDEX-CMIP6_mon.json"),
        dataset_table=os.path.join(table_dir, "CORDEX-CMIP6_remo_example.json"),
        grids_table=os.path.join(table_dir, "CORDEX-CMIP6_grids.json"),
        CORDEX_domain="EUR-11",
        time_units=None,
        allow_units_convert=True,
    )
    return filename

In [123]:
ds = cx.tutorial.open_dataset("remo_EUR-11_TEMP2_mon")
ds

<xarray.Dataset>
Dimensions:                     (rlon: 433, rlat: 433, height2m: 1, time: 12)
Coordinates:
  * rlon                        (rlon) float64 -28.93 -28.82 ... 18.49 18.6
  * rlat                        (rlat) float64 -23.93 -23.82 ... 23.49 23.6
  * height2m                    (height2m) float64 2.0
    lon                         (rlat, rlon) float64 ...
    lat                         (rlat, rlon) float64 ...
  * time                        (time) datetime64[ns] 2000-01-15 ... 2000-12-15
Data variables:
    TEMP2                       (time, height2m, rlat, rlon) float32 ...
    rotated_latitude_longitude  |S1 ...

In [126]:
ds.cf

Coordinates:
- CF Axes:   X, Y, Z, T: n/a

- CF Coordinates:   longitude: ['lat']
                    latitude: ['lat', 'lon']
                    vertical, time: n/a

- Cell Measures:   area, volume: n/a

- Standard Names: * grid_latitude: ['rlat']
                  * grid_longitude: ['rlon']
                    latitude: ['lon']
                    longitude: ['lat']

- Bounds:   n/a

Data Variables:
- Cell Measures:   area, volume: n/a

- Standard Names:   n/a

- Bounds:   n/a

In [153]:
wrf = xr.open_dataset("/Users/buntemey/python/py-cordex-data/wrfout_v2_Lambert.nc")
wrf

<xarray.Dataset>
Dimensions:    (Time: 13, south_north: 60, west_east: 73, bottom_top: 27,
                west_east_stag: 74, south_north_stag: 61, bottom_top_stag: 28,
                ext_scalar: 1, soil_layers_stag: 5)
Dimensions without coordinates: Time, south_north, west_east, bottom_top,
                                west_east_stag, south_north_stag,
                                bottom_top_stag, ext_scalar, soil_layers_stag
Data variables: (12/80)
    Times      (Time) |S19 ...
    LU_INDEX   (Time, south_north, west_east) float32 ...
    U          (Time, bottom_top, south_north, west_east_stag) float32 ...
    V          (Time, bottom_top, south_north_stag, west_east) float32 ...
    W          (Time, bottom_top_stag, south_north, west_east) float32 ...
    PH         (Time, bottom_top_stag, south_north, west_east) float32 ...
    ...         ...
    XLAND      (Time, south_north, west_east) float32 ...
    PBLH       (Time, south_north, west_east) float32 ...
    HFX        (Time, south_north, west_east) float32 ...
    QFX        (Time, south_north, west_east) float32 ...
    LH         (Time, south_north, west_east) float32 ...
    SNOWC      (Time, south_north, west_east) float32 ...
Attributes: (12/49)
    TITLE:                            OUTPUT FROM WRF V2.0 MODEL
    START_DATE:                      2000-01-24_12:00:00
    WEST-EAST_GRID_DIMENSION:        74
    SOUTH-NORTH_GRID_DIMENSION:      61
    BOTTOM-TOP_GRID_DIMENSION:       28
    GRIDTYPE:                        C
    ...                              ...
    ISWATER:                         16
    ISICE:                           24
    ISURBAN:                         1
    ISOILWATER:                      14
    MAP_PROJ:                        1
    MMINLU:                          USGS

In [154]:
wrf = xr.open_dataset(
    "/Users/buntemey/python/py-cordex-data/wrfout_d01_2005-08-28_00 00 00"
)
[a for a in wrf.attrs.keys() if "MAP" in a]

['MAP_PROJ', 'MAP_PROJ_CHAR']

In [149]:
wrf.MAP_PROJ_CHAR

'Mercator'

In [155]:
wrf.cf

Coordinates:
- CF Axes:   X, Y, Z, T: n/a

- CF Coordinates:   longitude: ['XLONG', 'XLONG_U', 'XLONG_V']
                    latitude: ['XLAT', 'XLAT_U', 'XLAT_V']
                    vertical, time: n/a

- Cell Measures:   area, volume: n/a

- Standard Names:   n/a

- Bounds:   n/a

Data Variables:
- Cell Measures:   area, volume: n/a

- Standard Names:   n/a

- Bounds:   n/a

In [131]:
wrf.XTIME

<xarray.DataArray 'XTIME' (Time: 4)>
array(['2005-08-28T00:00:00.000000000', '2005-08-28T03:00:00.000000000',
       '2005-08-28T06:00:00.000000000', '2005-08-28T09:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
    XTIME    (Time) datetime64[ns] 2005-08-28 ... 2005-08-28T09:00:00
Dimensions without coordinates: Time
Attributes:
    FieldType:    104
    MemoryOrder:  0  
    description:  minutes since 2005-08-28 00:00:00
    stagger: